In [13]:
from shared import *

In [14]:
wearables_df = get_wearable_types_df()

In [15]:
wearables_df.head(12)

,name,maxQuantity,rarityScoreModifier,NRG,AGG,SPK,BRN,NRG Effect,AGG Effect,SPK Effect,BRN Effect,slotNames,rarity
id,,,,,,,,,,,,,
1,Camo Hat,1000,1,0,1,0,0,,+,,,Head,common
2,Camo Pants,1000,1,0,1,0,0,,+,,,Body,common
3,MK2 Grenade,1000,1,0,1,0,0,,+,,,"Hand L, Hand R",common
4,Snow Camo Hat,500,2,0,1,1,0,,+,+,,Head,uncommon
5,Snow Camo Pants,500,2,0,1,1,0,,+,+,,Body,uncommon
6,M67 Grenade,500,2,0,2,0,0,,+,,,"Hand L, Hand R",uncommon
7,Marine Cap,300,5,0,2,0,1,,+,,+,Head,rare
8,Marine Jacket,300,5,0,2,0,1,,+,,+,Body,rare
9,Walkie Talkie,300,5,1,0,1,1,+,,+,+,"Hand L, Hand R",rare


In [16]:
wearables_df.describe()

,maxQuantity,rarityScoreModifier,NRG,AGG,SPK,BRN
count,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000
mean,339.894309,11.186992,0.032520,0.020325,0.052846,0.121951
std,333.655653,13.730258,1.540925,1.547741,1.278253,1.443269
min,5.000000,1.000000,-6.000000,-6.000000,-5.000000,-5.000000
25%,100.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,250.000000,5.000000,0.000000,0.000000,0.000000,0.000000
75%,500.000000,10.000000,1.000000,1.000000,0.000000,0.000000
max,1000.000000,50.000000,6.000000,6.000000,5.000000,6.000000


In [17]:
wearables_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246 entries, 1 to 315
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 246 non-null    object
 1   maxQuantity          246 non-null    int64 
 2   rarityScoreModifier  246 non-null    int64 
 3   NRG                  246 non-null    int64 
 4   AGG                  246 non-null    int64 
 5   SPK                  246 non-null    int64 
 6   BRN                  246 non-null    int64 
 7   NRG Effect           246 non-null    object
 8   AGG Effect           246 non-null    object
 9   SPK Effect           246 non-null    object
 10  BRN Effect           246 non-null    object
 11  slotNames            246 non-null    object
 12  rarity               246 non-null    object
dtypes: int64(6), object(7)
memory usage: 26.9+ KB


In [18]:
wearables_df.to_csv('db/wearable_types.csv')